#P0. Introducción -PLN

---

En la creación de redes neuronales necesitamos dos tipos de IA, para reconocer patrones o generar nuevos:

*   Las que no tienen memoria, identifica el patrón y ya!...ejemplo las de visión artificial
*   Las de memoria corta (Long Short Term Memory)...PLN
*   Las que requieren mucha memoria (aprenden casi todo...BERT)...PLN y visión artificial.



**Caso de estudio: generación de texto**

---


Cualquier dato que se necesite procesar (sonido, imágenes, texto) primero debe ser convertido en un tensor numérico, un paso llamado “vectorización” (One-hot Encoding y WordEmbedding) de datos (y en nuestro ejemplo previamente las letras deben ser pasadas a valores numéricos 

Para este ejemplo usaremos “*Character level language model*” propuesto por Andrej Karpathy en su artículo "*The Unreasonable Effectiveness of Recurrent Neural Networks*"(y parcialmente basado en su implementado en el tutorial "*Generate text with an RNN*" de la web de TensorFlow:

Consiste en darle a la RNN una palabra y se le pide que modele la distribución de probabilidad del siguiente carácter que le correspondería a la secuencia de caracteres anteriores:

Como ejemplo, supongamos que solo tenemos un vocabulario de cuatro letras posibles [“a”,”h”,”l”,”o”], y queremos entrenar a una RNN en la secuencia de entrenamiento “hola”. Esta secuencia de entrenamiento es, de hecho, una fuente de 3 ejemplos de entrenamiento por separado: La probabilidad de “o” debería ser verosímil dada el contexto de “h”, “l” debería ser verosímil en el contexto de “ho”, y finalmente “a” debería ser también verosímil dado el contexto de “hol”.


---
*   https://unipython.com/generacion-de-textos-con-inteligencia-artificial/
*    https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa (https://www.youtube.com/watch?v=kaQCdv46OBA&ab_channel=JeffHeaton)

#EJEMPLO 1: Prediciendo un texto (La biblioteca de label).

In [ ]:
!kill -9 -1 #reiniciando la maquina virtual

##P0. importando librerias

###Librerias genericas

In [ ]:
!pip list

In [9]:
import requests
import io
import os
import time

import numpy as np
import sys

#librerías para graficar
import matplotlib.pyplot as plt
                 
plt.rcParams['figure.figsize'] = (16, 9)  #ver graficas grandes 
plt.style.use('ggplot')
#guardar las imagenes y tablas en el cuaderno de jupyter
%matplotlib inline 

###librerias para DL (Deep Learning)

In [2]:
import tensorflow as tf

In [3]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.7.0
Eager mode:  True
GPU is available


Verificando los recursos de maquina para entrenar:

---
**CUDA (GPU-NVIDIA):**
CUDA es una plataforma de computación paralela y un modelo de programación que hace que el uso de una GPU para la computación de propósito general sea simple y elegante

In [ ]:
print("-------------------------------RAM------------------------------")
!cat /proc/meminfo  #cuanta memoria tenemos?
print("-----------------------------PROCESADOR-------------------------")
!cat /proc/cpuinfo  # que procesador tenemos?
print("-------------------------------CUDA-----------------------------")
!nvcc -V   # version de CUDA

###Activando la GPU

---
Logra aumentar la velocidad de entrenamiento en un 600% en PLN (RNN) y un 1000% en visión por computadores (CNN)


In [5]:
tf.device('/GPU:0')# OJO: esto es para que tensorflow utilice la GPU (aumenta la velocidad de entrenamiento en un 600%) 


##P1. Descarga y preprocesado de los datos

In [13]:
fileUrl     ='https://raw.githubusercontent.com/2HenryCardenas1/DeepLearning/main/Jupyter/PLN(Procesamiento_de_lenguajes_naturales)/Datasets/la_biblioteca_de_babel.txt'
fileContent = tf.keras.utils.get_file('la_biblioteca_de_babel.txt',fileUrl)
texto       = open(fileContent, 'rb').read().decode(encoding='utf-8')



##P2. pasar el texto a números

---
Sin importan el origen de la informacción (video, sonido, sensor, texto)...siempre debemos convertirlos en datos númericos.


In [14]:
print("[!] Pasando todo el texto a minuscula")
texto_min =texto.lower()
time.sleep(1)
print("[!] Quitando los caracteres especiales del texto")
texto_editado = texto_min.replace('\n', '').replace('\r','\n').replace('(','').replace(')','').replace('!','').replace('1','').replace('7','').replace('-','').replace('?','').replace('«','').replace('»','').replace('¿','').replace('¡','')
texto_final=texto_editado.replace('\n', '')
time.sleep(1)
print("Hecho !!")

[!] Pasando todo el texto a minuscula
[!] Quitando los caracteres especiales del texto
Hecho !!


In [15]:
chars = sorted(list(set(texto_final)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(chars)
print(char_to_int)

[' ', ',', '.', ':', ';', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', 'ñ']
{' ': 0, ',': 1, '.': 2, ':': 3, ';': 4, 'a': 5, 'b': 6, 'c': 7, 'd': 8, 'e': 9, 'f': 10, 'g': 11, 'h': 12, 'i': 13, 'j': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'q': 20, 'r': 21, 's': 22, 't': 23, 'u': 24, 'v': 25, 'x': 26, 'y': 27, 'z': 28, 'ñ': 29}


In [16]:
n_chars = len(texto_final)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))

En total hay 15631 caracteres, y el diccionario tiene un tamaño de 30 caracteres.


###P2.1 Dividimos el texto en secuencias:
---
Dividimos el texto en estas secuencias (adrede), convertimos los caracteres a números enteros usando nuestra tabla de búsqueda que preparamos anteriormente



In [ ]:
# preparar el conjunto de datos de los pares de entrada a salida codificados como enteros
seq_length = 50   #largo de las secciones de texto que usaremos para entrenar
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	print("Seq(",i,")=",texto_final[i:i + seq_length],"---->",raw_text[i + seq_length])
	seq_in 	= texto_final[i:i + seq_length]         					#secuencia de entrada
	seq_out = texto_final[i + seq_length]										#siguiente letra despues de la secuencia (la que el va a aprender)
	dataX.append([char_to_int[char] for char in seq_in])  # convertimos cada secuencia en numeros
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Se generaron ",format(n_patterns) ," secuencias texto de un tamaño de ",seq_length," caracteres" )
print("Como se ven los datos de X convertidos a números\n")
print(dataX)
print("\nComo se ven los datos de Y convertidos a números\n")
print(dataY)

##P3. preparar nuestros datos de entrenamiento

---


1.   Primero debemos transformar la lista de secuencias de entrada en la forma [muestras, pasos de tiempo, características] esperada por una red LSTM.
2.   Luego debemos cambiar la escala de los números enteros al rango de 0 a 1 para que los patrones sean más fáciles de aprender mediante la red LSTM que utiliza la función de activación sigmoidea de forma predeterminada.
3.   por ultimo necesitamos convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación one hot. Esto es para que podamos configurar la red para predecir la probabilidad de cada uno de los 54 caracteres diferentes en el vocabulario (una representación más fácil)







In [19]:
#transformar la lista X de secuencias de entrada en la forma [muestras (3091), pasos de tiempo, características]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalizar (cambiar la escala de los números enteros al rango de 0 a 1 )
X = X / float(n_vocab)
# convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación one hot.
y = tf.keras.utils.to_categorical(dataY)

##P4.Construcción del modelo RNN

---
definimos nuestro modelo LSTM: 
Aquí definimos una única capa LSTM oculta con 256 unidades de memoria. La red usa deserción con una probabilidad de 20. La capa de salida es una capa densa que usa la función de activación softmax para generar una predicción de probabilidad para cada uno de los 54 caracteres entre 0 y 1.


In [20]:
# define the LSTM model

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2])))    #creamos una capa con 256 unidades de memoria
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))            #Softmax convierte un vector de valores en una distribución de probabilidad para cada uno de los 39
                                                                              #Softmax se utiliza a menudo como la activación para la última capa de una red de clasificación
#utilizamos el algoritmo de optimización de ADAM para la velocidad
model.compile(loss='categorical_crossentropy', optimizer='adam')

###P4.1 Creando chekpoints

---

La red es lenta de entrenar (alrededor de 300 segundos por época) teniendo activa la GPU, ASí que crearemos CHECKPOINTS (puntos de control) para registrar todos los pesos de la red para archivar cada vez que se observe una mejora en la pérdida al final de la época. Usaremos el mejor conjunto de pesos (menor pérdida) para instanciar nuestro modelo generativo en la siguiente sección

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
# definimos  una carpeta para guardar los checkpoint
filepath="/content/drive/MyDrive/CheckPoints(prediccion de texto de un libro)/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

###P4.2 entrenando

In [ ]:
history = model.fit(X, y, epochs=1000, batch_size=128, verbose=1,callbacks=callbacks_list)

##P5.Generando texto con una red LSTM


---
Vamos a cargar el ultimo CHECKPOINT de entrenamiento y con el haremos MAGIA!!!


In [25]:
filename = "/content/drive/MyDrive/CheckPoints(prediccion de texto de un libro)/weights-improvement-947-0.0177.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

###P5.1 mapeo inverso (números a letras)
creamos un mapeo inverso que podamos usar para convertir los números enteros nuevamente en caracteres para que podamos entender las predicciones

In [26]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

###P5.2 hacer predicciones
La forma más sencilla de utilizar el modelo Keras LSTM para hacer predicciones es comenzar primero con una secuencia semilla como entrada, generar el siguiente carácter y luego actualizar la secuencia semilla para agregar el carácter generado al final y recortar el primer carácter. Este proceso se repite mientras queramos predecir nuevos caracteres (por ejemplo, una secuencia de 1000 caracteres de longitud).

In [27]:
# elige una semilla al azar
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de 10000 caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"o ,   n i n g u n a   m e n t e   r a z o n a b l e   p u e d e   d u d a r .   e l   h o m b r e ,"
 elimperfecto bibliotecario, puede ser obra del azar o de los demiurgos malevolos; eluniverso, con su elegante dotacion de anaqueles, de tomos enigmaticos, de infatigablesescaleras para el viajero y de letrinas para el bibliotecario sentado, solo puede ser obra deun dios. para percibir la distancia que hay entre lo divino y lo humano, basta comparar estos rudos simbolos tremulos que mi falible mano garabatea en la tapa de un libro, conlas letras organicas del interior: puntuales, delicadas, negrisimas, inimitablementesimetricas.el segundo: el numero de simbolos ortograficos es veinticinco. esa comprobacionpermitio, hace trescientos años, formular una teoria general de la biblioteca y resolversatisfactoriamente el problema que ninguna conjetura habia descifrado: la naturalezainforme y caotica de casi todos los libros. uno, que mi padre vio en un hexagono delcircuito quince nove

##P6.Mejorando la red (una LSTM más grande)

In [28]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(256))                                                                #agregaremos una segunda capa. 
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam')
#cambiamos el nombre de archivo de los pesos con puntos de control para que 
#podamos distinguir entre los pesos de esta red 
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

###P6.1 mejoramos el entrenamiento

---
aumentamos las epoch y disminuiremos el tamaño del lote de 128 a 64 para darle a la red más oportunidades de actualizarse y aprender.


In [29]:
#los tiempos de entrenamiento aumentaran al doble que en la versión anterior
model.fit(X, y, epochs=50, batch_size=32, callbacks=callbacks_list)

###P6.2 haciendo predicciones

In [30]:
# elige una semilla al azar
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"q u e   d a   t o d a   l a v u e l t a   d e   l a s   p a r e d e s ;   p e r o   s u   t e s t i"
doen los hombres de libros.ee eormato una soda lavualis del uo hexagono del cumto de esefuncionario remoto. muchos peregrinadas en las letrinas, con unos discos dem comentario de ese evangelio, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia de esoscatalogos, la demostracion de la f

##P7. exportar modelo RNN

In [31]:
!pip install h5py

In [42]:
from keras.models import model_from_json
import os
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("modelRNN_cuentos.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/MyDrive/Models(RNN)/modeloRNN_cuento_la_biblioteca_de_babel.hdf5")
model.save('/content/drive/MyDrive/Models(RNN)/modelRNN__cuento_la_biblioteca_de_babel_v1_h5.h5')
print("modelo salvado en disco")

modelo salvado en disco


###P7.1 cargando un modelo

In [37]:
# Recrea exactamente el mismo modelo solo desde el archivo
new_model = tf.keras.models.load_model('/content/modelRNN__cuento_la_biblioteca_de_babel_v1_h5.h5')

In [38]:
chars = sorted(list(set("ninguna mente razonable puede dudar")))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))
pattern = dataX[5]
print(pattern)

En total hay 15876 caracteres, y el diccionario tiene un tamaño de 16 caracteres.
[13, 25, 9, 21, 22, 18, 0, 20, 24, 9, 0, 18, 23, 21, 18, 22, 0, 15, 15, 5, 16, 5, 17, 0, 15, 5, 0, 6, 13, 6, 15, 13, 18, 23, 9, 7, 5, 0, 22, 9, 0, 7, 18, 16, 19, 18, 17, 9, 0, 8]


In [41]:
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = new_model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"a   d i s t r i b u c i o n   d e   l a s   g a l e r i a s   e s   i n v a r i a b l e .   v e i n"
ebio dasagooo laromalam; anio deebanadan daxagsoebres y qo pilpeeser; eletoe anes p mi miopegmesa ened q li eesorabenas, po hredi, fi heguil desari mo te jo freo, pi mi mi lievera l li melilebiean devaroaloe s mi peliamear; cayagso li midbresio faguaro mue mi megular m euedsearoo fagagsoras.m m mimpearo anasio faroras m qi mi mi milpearo q mi mi mi mreeso s mi milpefsesa eleduerar, earoiribnebroo davagooo lamimabes detanmabas domoralidoebreo, ieo, ni me seloe  lo frearo anis deebananan n mi memilen q qo lideeeeser; y gamomamam;bas docara liliiioebanoes;r des he mi li fimoaf q li lilpesmas davagsoo laromalam; anioi canagrororid s li lidbmearoo q mieperman m euedeees. ca piamisisiribccua y lamomebanoearoo v lamomecacooo davag t mi miever, mi ne milueano daxagoro laromalamaaariiiiepitibde, moe eoso v lamamearoableo;n n datoribarooalas,m moeeeeler; hoeduer; v to eioo fatoraboeoas,

#Ejemplo 2: generar texto de cuentos, usando Keras

##P0. importar librerias

In [ ]:
!pip install tensorflow-gpu

In [44]:
import tensorflow as tf
import numpy as np
import os
import time
import sys

uso de GPU para entrenar en tensorflow

---
https://medium.com/analytics-vidhya/solution-to-tensorflow-2-not-using-gpu-119fb3e04daa


In [45]:
print("Version: ", tf.__version__)
#print("Eager mode: ", tf.executing_eagerly())
print("GPU esta", "disponible" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print("Dispositivos disponibles: ", tf.config.list_physical_devices())

Version:  2.7.0
GPU esta disponible
Dispositivos disponibles:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [46]:
#tf.device('/gpu:0') #activando la CPU
tf.device('/GPU:0') #activando la GPU 

##P0. Descarga y preprocesado de los datos

In [47]:
fileDL= tf.keras.utils.get_file('la_biblioteca_de_babel.txt','https://raw.githubusercontent.com/2HenryCardenas1/DeepLearning/main/Jupyter/PLN(Procesamiento_de_lenguajes_naturales)/Datasets/la_biblioteca_de_babel.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')
texto = texto.lower()

##P1. entendiendo el texto

Normalizando el texto

In [48]:
print("[!] Pasando todo el texto a minuscula")
texto_min =texto.lower()
time.sleep(1)
print("[!] Quitando los caracteres especiales del texto")
texto_editado = texto_min.replace('\n', '').replace('\r','\n').replace('(','').replace(')','').replace('!','').replace('1','').replace('7','').replace('-','').replace('?','').replace('«','').replace('»','').replace('¿','').replace('¡','')
texto_final=texto_editado.replace('\n', '')
time.sleep(1)
print("Hecho !!")

[!] Pasando todo el texto a minuscula
[!] Quitando los caracteres especiales del texto
Hecho !!


In [49]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto_final)))
vocab = sorted(set(texto_final))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:15631 caracteres
el texto esta compuesto de estos :30 caracteres
[' ', ',', '.', ':', ';', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', 'ñ']


##P2. pasar el texto a números

---
as redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres

In [51]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto_final])
print('texto: {}'.format(repr(texto_final[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'el universo que otros llaman la biblioteca se compone de un numero indefinido, ytal vez infinito, de'
array([ 9, 15,  0, 24, 17, 13, 25,  9, 21, 22, 18,  0, 20, 24,  9,  0, 18,
       23, 21, 18, 22,  0, 15, 15,  5, 16,  5, 17,  0, 15,  5,  0,  6, 13,
        6, 15, 13, 18, 23,  9,  7,  5,  0, 22,  9,  0,  7, 18, 16, 19, 18,
       17,  9,  0,  8,  9,  0, 24, 17,  0, 17, 24, 16,  9, 21, 18,  0, 13,
       17,  8,  9, 10, 13, 17, 13,  8, 18,  1,  0, 27, 23,  5, 15,  0, 25,
        9, 28,  0, 13, 17, 10, 13, 17, 13, 23, 18,  1,  0,  8,  9])


##P3. preparar los datos para ser usados en la RNN

In [52]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=150
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'el universo que otros llaman la biblioteca se compone de un numero indefinido, ytal vez infinito, de galerias hexagonales, con vastos pozos de ventilac'
'ion en el medio,cercados por barandas bajisimas. desde cualquier hexagono se ven los pisos inferiores ysuperiores: interminablemente. la distribucion d'
'e las galerias es invariable. veinteanaqueles, a cinco largos anaqueles por lado, cubren todos los lados menos dos; su altura,que es la de los pisos, e'
'xcede apenas la de un bibliotecario normal. una de las caras libresda a un angosto zaguan, que desemboca en otra galeria, identica a la primera y a tod'
'as. aizquierda y a derecha del zaguan hay dos gabinetes minusculos. uno permite dormir depie; otro, satisfacer las necesidades finales. por ahi pasa la'
' escalera espiral, que se abismay se eleva hacia lo remoto. en el zaguan hay un espejo, que fielmente duplica lasapariencias. los hombres suelen inferi'
'r de ese espejo que la biblioteca no es infinita silo fuera realmente a que

###P3.1 separar los datos en agrupamientos (batches)

In [53]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'el universo que otros llaman la biblioteca se compone de un numero indefinido, ytal vez infinito, de galerias hexagonales, con vastos pozos de ventila'
Target data:  'l universo que otros llaman la biblioteca se compone de un numero indefinido, ytal vez infinito, de galerias hexagonales, con vastos pozos de ventilac'


In [54]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


##P4.Construcción del modelo RNN

---
Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.


In [ ]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=lossy,
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

###P4.1 Creando chekpoints

---
una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.

---
los crearemos en google drive para mejorar la capacidad de reentrenamiento de la red


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [56]:
checkpoint_dir='/content/drive/MyDrive/CheckPoints(generando texto de cuentos)/'#direccion de la carpeta checkpoint
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')


###P4.2 entrenando

####P4.2a entrenando para usar chekpoints

In [ ]:
EPOCHS=1000 #Segun la cantidad de epoch que queramos seguir entrenando
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])


#####4.2a-1 entrenando desde un checkpoint

---
Desde la carpeta que optamos guardar los checkpoints

*   el archivo .data es el archivo que contiene nuestras variables de entrenamiento y vamos a ir tras él.
*   el archivo checkpoint, simplemente mantiene un registro de los últimos archivos de punto de control guardados




In [ ]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model2=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpointsV3/cp_0030.ckpt


In [ ]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model2.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history_V2=model2.fit(dataset, 
                    epochs=100, 
                    verbose=1,
                    callbacks=[cp_callback])

####P4.2b entrenando con tensorboard (opcional)

#####Activando TENSORBOARD 

---
(DASHBOARD para ver el proceso de entrenamiento)

In [ ]:
# You can change the directory name
LOG_DIR = '/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/PLN/Redes neuronales recurrentes (RNN)/checkpoints'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=BATCH_SIZE,
                         write_images=True)

#####Fit

In [ ]:

#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

##P5. Generando texto nuevo usando la RNN

In [64]:
#creamos un modelo tomando como base el ultimo checkpoint
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))##tomamos el ultimo checkpoint
model.build(tf.TensorShape([1,None]))

In [65]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.3  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


###P5.1 generando texto 

In [66]:
print(generate_text(model, start_string=u"la colonia europea"))

la colonia european que las parie el desprico de aque en lugares remotos los corredores y escaleras y hexagonos puedeninconcebiblemente cesar, lo cual es absurdo. quienes la imaginan sin limites, olvide catalogos falsos, la demostracion de la falacia de esoscatalogos, la demostracion de la falacia del catalogo verdadero, el evangelio gnostico debasiguna del cutron todas las pombres scoren queeno deplicado se alobra no un selor en esas.otrados enoraque el dios hexagonoles, invar able denes, de nuestra intuicionde 


##P6.exportando modelo

---
Guardamos y Serializamos el Modelo (con esto ya podemos vender nuestro modelo de predicción de texto según lo aprendido por nuestra RNN).


In [67]:
from keras.models import model_from_json
import os
dir_export= '/content/drive/MyDrive/Models(RNN)' #Ruta del directorio el cual vamos a guardar el modelo
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_LaBibliotecaDeBabel_json.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_LaBibliotecaDeBabel_pesos.hdf5'))
model.save(os.path.join(dir_export,'RNN_LaBibliotecaDeBabel_model.h5'))
print("modelo salvado en Drive de google")

modelo salvado en Drive de google


##P7.Cargando un modelo serializado

###P7.1 descargamos el modelo usando wget

Ponemos la url del

In [72]:
!wget 'https://github.com/2HenryCardenas1/DeepLearning/blob/main/Jupyter/PLN(Procesamiento_de_lenguajes_naturales)/Models/RNN_LaBibliotecaDeBabel_model.h5?raw=true' -O RNN_LaBibliotecaDeBabel_model.h5


--2021-11-19 16:25:04--  https://github.com/2HenryCardenas1/DeepLearning/blob/main/Jupyter/PLN(Procesamiento_de_lenguajes_naturales)/Models/RNN_LaBibliotecaDeBabel_model.h5?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/2HenryCardenas1/DeepLearning/raw/main/Jupyter/PLN(Procesamiento_de_lenguajes_naturales)/Models/RNN_LaBibliotecaDeBabel_model.h5 [following]
--2021-11-19 16:25:05--  https://github.com/2HenryCardenas1/DeepLearning/raw/main/Jupyter/PLN(Procesamiento_de_lenguajes_naturales)/Models/RNN_LaBibliotecaDeBabel_model.h5
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/2HenryCardenas1/DeepLearning/main/Jupyter/PLN(Procesamiento_de_lenguajes_naturales)/Models/RNN_LaBibliotecaDeBabel_model.h5 [following]
--2021-11-19 16:25:05--  https

####P7.1a descargamos el modelo usando PYRIND & URLLIB (OPCIONAL)

In [76]:
!pip install pyprind

In [77]:
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()

import urllib.request
url_github_Model='https://github.com/2HenryCardenas1/DeepLearning/blob/main/Jupyter/PLN(Procesamiento_de_lenguajes_naturales)/Models/RNN_LaBibliotecaDeBabel_model.h5?raw=true'
urllib.request.urlretrieve(url_github_Model,
                           'RNN_LaBibliotecaDeBabel_model2.h5', 
                           reporthook)

100% | 60 MB | 3.53 MB/s | 17 segundos transcurrido

('RNN_LaBibliotecaDeBabel_model2.h5',
 <http.client.HTTPMessage at 0x7f35e1178c50>)

###P7.2 instanciamos el modelo descargado

In [80]:
from tensorflow import keras

In [81]:
new_model = keras.models.load_model('/content/RNN_LaBibliotecaDeBabel_model2.h5') #Nombre del modelo descargado

In [81]:
print(generate_text(new_model, start_string=u"colombia es un pais ")) #palabra 